## 모듈 import

In [1]:
#모듈 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from pycaret.classification  import *
from sklearn.tree import DecisionTreeClassifier

# feature selection
import sys, warnings
if not sys.warnoptions: warnings.simplefilter("ignore")
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import cross_val_score
from tqdm import tqdm

#Scailing
from sklearn.preprocessing import PowerTransformer

#Label Encoding
from sklearn.preprocessing import LabelEncoder

# oversampling
import random
# w2v
from gensim.models import word2vec

In [2]:
#삡운드
import winsound as sd
# 삡싸운드
def beepsound():
    fr = 2000   # range : 37 ~ 32767
    du = 1000 # 1000 ms ==1second
    sd.Beep(fr, du) # winsound.Beep(frequency, duration)

## data 불러오기

In [3]:
# 데이터가 저장된 경로 설정
data_path = '../daplatformers2021/'

# 데이터 불러오기
train = pd.read_csv(data_path + 'trainset.csv')
test = pd.read_csv(data_path + 'testset_final.csv')
submission = pd.read_csv(data_path + 'sample_submission_final.csv')

In [4]:
train = train.drop(columns=['매니저최초가입일', '매니저최초서비스일', '매니저주소'])

### 전처리

In [2]:
data = pd.concat([train,test]).reset_index(drop=True)

- 결측치 채움

In [6]:
### 결측치 제거

# 결측치 0으로 채움
data['청소교육이수여부'] = data['청소교육이수여부'].fillna(0)
data['CS교육이수여부'] = data['CS교육이수여부'].fillna(0)
data['부재중여부'] = data['부재중여부'].fillna(0)
# 결측치 최빈값으로 채움
data['매니저이동방법'] = data['매니저이동방법'].fillna(data['매니저이동방법'].mode()[0])
data['매니저사용휴대폰'] = data['매니저사용휴대폰'].fillna(data['매니저사용휴대폰'].mode()[0])
data['고객가입일'] = data['고객가입일'].fillna(data['고객가입일'].mode()[0]) #데이터 바뀌고 test에 생긴 에러
data['결재형태'] = data['결재형태'].fillna(data['결재형태'].mode()[0]) #데이터 바뀌고 test에 생긴 에러

data['접수시각'] = data['접수시각'].fillna(data['접수시각'].mode()[0])
data['반려동물'] = data['반려동물'].fillna('없음')
# 결측치 판단하여 없음으로 채움
data['우선청소'] = data['우선청소'].fillna('없음')

### 평수 평균 확인 -> 31.xx -> 30평대로 채움
# (10*len(data.query('평수 == "10평대"')) + 20*len(data.query('평수 == "20평대"')) + 
#  30*len(data.query('평수 == "30평대"')) + 40*len(data.query('평수 == "40평대이상"')))/ \
# (len(data.query('평수 == "10평대"'))+len(data.query('평수 == "20평대"'))+
#  len(data.query('평수 == "30평대"'))+len(data.query('평수 == "40평대이상"')))
data['평수'] = data['평수'].fillna('30평대')

In [7]:
#test와 train의 구조를 맞춰줌
data['매니저생년월일'] = data['매니저생년월일'].astype('str').apply(lambda x: x[0:4])
# 새로운 피처 만들기
data['매니저나이'] = data['매니저생년월일'].apply(lambda x: 2021-int(x))
data['매니저나이대'] = data['매니저나이'].apply(lambda x: f'{str(x)[0]}0대')

In [9]:
# 저장해놓기
data_SEQ = data.SEQ
# 인코딩을 위하여 제거 / 이후 나중에 병합함
y_data = data['매칭성공여부']
data = data.drop(columns = ['SEQ','매칭성공여부'])

In [11]:
#라벨인코딩

#범주형변수 수동으로 분리
data[['장기서비스여부','기존고객여부','부재중여부','쿠폰사용여부','매니저성별',
         'CS교육이수여부','청소교육이수여부','부재중서비스가능여부','추천인여부']
     ] = data[['장기서비스여부','기존고객여부','부재중여부','쿠폰사용여부','매니저성별',
         'CS교육이수여부','청소교육이수여부','부재중서비스가능여부','추천인여부'
     ]].astype('str')

# 범주형 변수와 수치형 변수를 분리
cat_features = data.select_dtypes(include=['object']).columns.to_list()
num_features = data.select_dtypes(exclude='object').columns.to_list() 

#수치형변수 scailing
data[num_features] = PowerTransformer(standardize=True).fit_transform(data[num_features])

In [12]:
#위에서 떼어놨던 피처 병합
data = pd.concat([data_SEQ, data],axis=1)
data = pd.concat([data, y_data],axis=1)

In [13]:
#train과 test를 분리
train = data.iloc[:23009,:]
test = data.iloc[23009:,:]
test = test.drop(columns = ['매칭성공여부']).reset_index(drop=True)

### W2V

In [14]:
# 오버샘플링
def oversample(x, n):
    lst = []
    for i in x:
        tmp = []
        for j in range(n):
            random.shuffle(i)
            tmp += list(i)
        lst.append(tmp)
    return lst

In [15]:
cat_features_w2v = []
# 모든 범주형 피처를 w2v
for level in tqdm(cat_features):
    # W2V 학습을 하기에는 데이터(즉 corpus)가 부족하여 oversampling을 수행
    sentences = [ list(data.query('SEQ == @id')[level].values) for id in data['SEQ'].unique() ]

    # oversampling
    w2v_input = oversample(sentences, 5)

    max_features = 300 # 문자 벡터 차원 수
    min_word_count = 1 # 최소 문자 수
    num_workers = 4 # 병렬 처리 스레드 수
    context = 3 # 문자열 창 크기
    downsampling = 1e-3 # 문자 빈도수 Downsample

    # 모델 학습
    w2v = word2vec.Word2Vec(sentences = w2v_input,
                            vector_size = 30,
                            window = 3,
                            min_count = 1,
                            sg = 1).wv
    # 필요없는 메모리 unload
    w2v.init_sims(replace=True)

    #train
    train_mean_vector = []

    for words in sentences:
            tmp = np.zeros(30)             # 다음 customer ID에 대한 vector를 계산하기 전 0으로 초기화
            cnt = 0
            for word in words:
                try:
                    tmp += w2v[word]
                    cnt += 1
                except:
                    pass
            tmp /= cnt                      # customer ID 에 있는 아이템 갯수로 전체 벡터를 mean해줌  
            train_mean_vector.append(tmp)

    train_mean_vector = np.array(train_mean_vector)

    locals()[f'data_{level}'] = pd.DataFrame(train_mean_vector)
    cat_features_w2v.append(locals()[f'data_{level}'])

100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [27:19<00:00, 60.71s/it]


In [16]:
w2v_data = pd.concat(cat_features_w2v,axis=1)

In [1]:
data = pd.concat([data_SEQ, w2v_data],axis=1)
data = pd.concat([w2v_data, y_data],axis=1);data.head(3)

In [18]:
# 열 중복 해결
col_name= [i for i in range(len(data.columns)-1)]
col_name.append('매칭성공여부')
data.columns = col_name

In [19]:
#w2v 데이터 저장
data.to_csv('w2v_data.csv', index=False)

In [51]:
beepsound(),beepsound(),beepsound()

(None, None, None)

# END